# 1. LoRA - BERT 모델을 사용한 PEFT

In [ ]:
from huggingface_hub import login

# 위젯 대신 login 함수 안에 토큰을 직접 넣어서 로그인합니다.
# "hf_..." 부분에 복사해둔 토큰을 따옴표 안에 넣어주세요.
login(token="hf_...")

# 만약 모델을 업로드(저장)까지 해야 한다면 아래처럼 옵션을 추가할 수 있습니다.
# login(token="hf_...", write_permission=True)

## BERT 모델 로드

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModel,
    Trainer,
    TrainingArguments
)

# BERT 로드
base_model = AutoModel.from_pretrained(
    "klue/bert-base",
    num_labels=2
)

base_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

## 데이터셋 로드

In [4]:
from datasets import load_dataset

# 예: Hugging Face Hub에 올라가 있는 NSMC 변환본
dataset = load_dataset("nsmc")   # <- 여기만 바꾸기

print(dataset)
print(dataset["train"][0])


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})
{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}


## 토크나이저 설정


In [ ]:
# 토크나이저

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

def preprocess(example):
    return tokenizer(
        example["document"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

    tokenized = dataset.map(preprocess, batched=True)
    

HFValidationError: Repo id must use alphanumeric chars, '-', '_' or '.'. The name cannot start or end with '-' or '.' and the maximum length is 96: ''.